In [1]:
import re
from typing import Iterator

import numpy as np

# enwiki-20210520-page Stats
- 6265 INSERT lines
- 6,301,566 pages in ns0
- max page_id: 67,715,788 (unsigned 32 bit int can represent 4,294,967,295, so it is safe to use np.uint32 for page_id)

## INSERT entry format
```python
(10,0,'AccessibleComputing','',1,0,0.33167112649574004,'20210427193048','20210126121900',1002250816,111,'wikitext',NULL)
page_id, page_ns, page_title
```
### Relevant groups:
- 1: page_id
- 2: page_ns
- 3: page_title
- 5: page_is_redirect (1: yes, 0: no)

In [2]:
line_start = re.compile(b"INSERT INTO (.*?) VALUES ")
item_ex = re.compile(b"\((.*?),(.*?),'(.*?)',(.*?),(.*?),(.*?)\)")
pages = []

In [3]:
def get_line_iterator(line) -> Iterator:
    if line_start.search(line) is not None:
        item_iter = item_ex.finditer(line)
        return item_iter
    else:
        return None


def find_pages_in_ns(line_iter, ns=0):
    for entry in line_iter:
        ns_b_string = bytes(f"{ns}", 'utf-8')
        if entry.group(2) == ns_b_string and entry.group(5) == b'0':
            pages.append([entry.group(1), entry.group(3)])
            if len(pages) % 500000 == 0:
                print(f"matched {len(pages)} pages")

       


def process_line(line, ns=0):
    line_iter = get_line_iterator(line)
    if line_iter is None:
        return 0
    find_pages_in_ns(line_iter, ns=ns)

In [4]:
namespace = 1
with open("data/enwiki-20210520-page.sql", "rb") as file:
    for line in file:
        if line_start.search(line) is not None:
            process_line(line, ns=namespace)

matched 500000 pages
matched 1000000 pages
matched 1500000 pages
matched 2000000 pages
matched 2500000 pages
matched 3000000 pages
matched 3500000 pages
matched 4000000 pages
matched 4500000 pages
matched 5000000 pages
matched 5500000 pages
matched 6000000 pages
matched 6500000 pages
matched 7000000 pages


In [8]:
np_array=np.array(pages)

In [7]:
for page in pages:
    if page[1] == b'Anarchism':
        print(page)

[b'416541', b'Anarchism']


In [5]:
len(pages)

7043782

In [9]:
np.save(f"ns{namespace}_pages", np_array)

In [12]:
val = 5
bytes(f"{val}", 'utf-8')

b'5'

In [2]:
np_pages = np.load("ns0_pages.npy")

In [6]:
for i in range(len(np_pages)):
    for page in np_pages:
        first_match = True
        if first_match and (np_pages[i][0] == page[0]):
            first_match = False
        if not first_match and (np_pages[i][0] == page[0]):    
            print("Duplicate page_id found:")
            print(np_pages[i][0])
            print(np_pages[i][1], page[1])

Duplicate page_id found:
b'12'
b'Anarchism' b'Anarchism'
Duplicate page_id found:
b'12'
b'Anarchism' b'Anarchism'
Duplicate page_id found:
b'25'
b'Autism' b'Autism'
Duplicate page_id found:
b'25'
b'Autism' b'Autism'
Duplicate page_id found:
b'39'
b'Albedo' b'Albedo'
Duplicate page_id found:
b'39'
b'Albedo' b'Albedo'
Duplicate page_id found:
b'272'
b'AcademicElitism' b'AcademicElitism'
Duplicate page_id found:
b'272'
b'AcademicElitism' b'AcademicElitism'
Duplicate page_id found:
b'290'
b'A' b'A'
Duplicate page_id found:
b'290'
b'A' b'A'
Duplicate page_id found:
b'303'
b'Alabama' b'Alabama'
Duplicate page_id found:
b'303'
b'Alabama' b'Alabama'
Duplicate page_id found:
b'305'
b'Achilles' b'Achilles'
Duplicate page_id found:
b'305'
b'Achilles' b'Achilles'
Duplicate page_id found:
b'307'
b'Abraham_Lincoln' b'Abraham_Lincoln'
Duplicate page_id found:
b'307'
b'Abraham_Lincoln' b'Abraham_Lincoln'
Duplicate page_id found:
b'308'
b'Aristotle' b'Aristotle'
Duplicate page_id found:
b'308'
b'Aristo

KeyboardInterrupt: 